In [1]:
from __future__ import print_function

#import os
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
#os.environ["CUDA_VISIBLE_DEVICES"] = ""

import time
from PIL import Image
import numpy as np

from keras import backend
from keras.models import Model
from keras.applications.vgg16 import VGG16

from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave

import os
cwd = os.getcwd()

Using TensorFlow backend.


In [1]:
def img_merge(iterations,cnt_layers,sty_layers, cnt_name, sty_name, height,
              width, content_weight, style_weight, total_variation_weight = 1.0, sty_rig = 0.1):

    content_image_path = cwd + '/content/' + cnt_name + '.jpg'
    content_image = Image.open(content_image_path)
    content_image = content_image.resize((width, height))

    style_image_path = cwd + '/style/' + sty_name + '.jpg'
    style_image = Image.open(style_image_path)
    style_image = style_image.resize((width, height))
    
    content_array = np.asarray(content_image, dtype='float32')
    content_array = np.expand_dims(content_array, axis=0)
    style_array = np.asarray(style_image, dtype='float32')
    style_array = np.expand_dims(style_array, axis=0)
    
    content_array[:, :, :, 0] -= 103.939
    content_array[:, :, :, 1] -= 116.779
    content_array[:, :, :, 2] -= 123.68
    content_array = content_array[:, :, :, ::-1]
    style_array[:, :, :, 0] -= 103.939
    style_array[:, :, :, 1] -= 116.779
    style_array[:, :, :, 2] -= 123.68
    style_array = style_array[:, :, :, ::-1]
    
    content_image = backend.variable(content_array)
    style_image = backend.variable(style_array)
    combination_image = backend.placeholder((1, height, width, 3))
    input_tensor = backend.concatenate([content_image,
                                        style_image,
                                        combination_image], axis=0)
    model = VGG16(input_tensor=input_tensor, weights='imagenet', include_top=False)
    layers = [layer.output for layer in model.layers]
    
    loss = backend.variable(0.)
    def content_loss(content, combination):
        return backend.sum(backend.square(combination - content))

    for i in cnt_layers:
        layer_features = layers[i]
        content_features = layer_features[0, :, :, :]
        combination_features = layer_features[2, :, :, :]
        sl = content_loss(content_features, combination_features)
        loss += (content_weight / len(cnt_layers)) * sl
        
    def gram_matrix(x):
        features = backend.batch_flatten(backend.permute_dimensions(x, (2, 0, 1)))
        gram = backend.dot(features, backend.transpose(features))
        return gram

    def style_loss(style, combination):
        S = gram_matrix(style)
        C = gram_matrix(combination)
        channels = 3
        size = height * width
        return backend.sum(backend.square(S - C)) / (4. * (channels ** 2) * (size ** 2)) #+ 0.005*content_loss(style, combination)

    for i in sty_layers:
        layer_features = layers[i]
        style_features = layer_features[1, :, :, :]
        combination_features = layer_features[2, :, :, :]
        sl = style_loss(style_features, combination_features)  + \
             sty_rig*(content_weight/ len(sty_layers))*content_loss(style_features, combination_features)
        loss += (style_weight / len(sty_layers)) * sl
        
    grads = backend.gradients(loss, combination_image)
    
    outputs = [loss]
    outputs += grads
    f_outputs = backend.function([combination_image], outputs)

    def eval_loss_and_grads(x):
        x = x.reshape((1, height, width, 3))
        outs = f_outputs([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        return loss_value, grad_values

    class Evaluator(object):

        def __init__(self):
            self.loss_value = None
            self.grads_values = None

        def loss(self, x):
            assert self.loss_value is None
            loss_value, grad_values = eval_loss_and_grads(x)
            self.loss_value = loss_value
            self.grad_values = grad_values
            return self.loss_value

        def grads(self, x):
            assert self.loss_value is not None
            grad_values = np.copy(self.grad_values)
            self.loss_value = None
            self.grad_values = None
            return grad_values

    evaluator = Evaluator()
    
    x = np.random.uniform(0, 255, (1, height, width, 3)) - 128.
    X = np.zeros((iterations,height,width,3))
    for i in range(iterations):
        #print('Start of iteration', i)
        start_time = time.time()
        x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
        X[i,...] = x.reshape((height, width, 3))
        #print('Current loss value:', min_val)
        end_time = time.time()
        #print('Iteration %d completed in %ds' % (i, end_time - start_time))
        
    x = x.reshape((height, width, 3))
    x = x[:, :, ::-1]
    x[:, :, 0] += 104
    x[:, :, 1] += 117
    x[:, :, 2] += 124
    x = np.clip(x, 0, 255).astype('uint8')
    print(cnt_layers, sty_layers)
    return(Image.fromarray(x).resize((width, height)))

In [30]:
imsave('results/' + cnt_name + '-' + sty_name + str(content_weight) + '.jpg', final_img)

NameError: name 'cnt_name' is not defined

In [36]:
import random
cnt_name = 'mine'
sty_name = 'graf'
for i in range(20):
    cnt_layers = random.sample(range(18),5)
    sty_layers = random.sample(range(18),5)
    final_img = img_merge(iterations = 10, cnt_layers=cnt_layers,sty_layers=sty_layers, cnt_name = cnt_name, 
                          sty_name = sty_name, height = 300, width = 900, content_weight = 0.05, style_weight = 40.0,
                          total_variation_weight = 1.0, sty_rig = 0.1)
    imsave('results/' + cnt_name + '-' + sty_name + str(i) + '.jpg', final_img)

TypeError: img_merge() got an unexpected keyword argument 'sty_rig'

In [32]:
for i in np.linspace(-1,2,5):
    cnt_layers = random.sample(range(18),5)
    sty_layers = random.sample(range(18),5)
    final_img = img_merge(iterations = 10, cnt_layers=[8, 5, 6, 12, 11],sty_layers=[10, 5, 11, 2, 8], cnt_name = cnt_name, 
                          sty_name = sty_name, height = 300, width = 900, content_weight = 10**i, style_weight = 40.0,
                          total_variation_weight = 1.0)
    imsave('results/' + cnt_name + '-' + sty_name + str(i) + 'a.jpg', final_img)

[8, 5, 6, 12, 11] [10, 5, 11, 2, 8]
[8, 5, 6, 12, 11] [10, 5, 11, 2, 8]
[8, 5, 6, 12, 11] [10, 5, 11, 2, 8]
[8, 5, 6, 12, 11] [10, 5, 11, 2, 8]
[8, 5, 6, 12, 11] [10, 5, 11, 2, 8]


In [17]:
final_img = img_merge(iterations = 10, cnt_layers=[8, 5, 6, 12, 11],sty_layers=[10,5,11,2,8], 
                      cnt_name = 'mine', sty_name = 'graf', height = 600, width = 1500, content_weight = 0.075, 
                      style_weight = 40.0, sty_rig = 0.0, total_variation_weight =1)
#
final_img

[8, 5, 6, 12, 11] [10, 5, 11, 2, 8]


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [18]:
imsave('results/' + 'mine' + '-' + 'graf' + 'final2.jpg', final_img)